In [129]:
%run data_loader.py

Data loaded


# Building Recommender Systems for Movie Rating Prediction

In this assignment, we will build a recommender systems that predict movie ratings. [MovieLense](https://grouplens.org/datasets/movielens/) has currently 25 million user-movie ratings.  Since the entire data is too big, we use  a 1 million ratings subset [MovieLens 1M](https://www.kaggle.com/odedgolden/movielens-1m-dataset), and we reformatted the data to make it more convenient to use.

In [130]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx
from tqdm import tqdm

In [131]:
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [132]:
from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

### Starter codes
Now, we will be building a recommender system which has various techniques to predict ratings. 
The `class RecSys` has baseline prediction methods (such as predicting everything to 3 or to average rating of each user) and other utility functions. `class ContentBased` and `class Collaborative` inherit `class RecSys` and further add methods calculating item-item similarity matrix. You will be completing those functions using what we learned about content-based filtering and collaborative filtering.

`RecSys`'s `rating_matrix` method converts the (user id, movie id, rating) triplet from the train data (train data's ratings are known) into a utility matrix for 6040 users and 3883 movies.    
Here, we create the utility matrix as a dense matrix (numpy.array) format for convenience. But in a real world data where hundreds of millions of users and items may exist, we won't be able to create the utility matrix in a dense matrix format (For those who are curious why, try measuring the dense matrix self.Mr using .nbytes()). In that case, we may use sparse matrix operations as much as possible and distributed file systems and distributed computing will be needed. Fortunately, our data is small enough to fit in a laptop/pc memory. Also, we will use numpy and scipy.sparse, which allow significantly faster calculations than calculating on pandas.DataFrame object.    
In the `rating_matrix` method, pay attention to the index mapping as user IDs and movie IDs are not the same as array index.

In [133]:
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        self.Mm=None
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies)))
        
    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(self.data.train.rating)
        
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())


    def predict_everything_to_3(self):
        """
        Predict everything to 3 for the test data
        """
        # Generate an array with 3s against all entries in test dataset
        # your code here
        return np.ones(len(self.data.test.rating))*3
        
        
    def predict_to_user_average(self):
        """
        Predict to average rating for the user.
        Returns numpy array of shape (#users,)
        """

        # Generate an array as follows:
        # 1. Calculate all avg user rating as sum of ratings of user across all movies/number of movies whose rating > 0
        # 2. Return the average rating of users in test data
        # your code here

        mean_users = { ind: user_ratings[user_ratings>0].mean()  for ind, user_ratings in enumerate(self.Mr)}
        mean = [mean_users[self.uid2idx[uid]] for uid in self.data.test["uID"]]
        return np.array(mean)
    
    def predict_from_sim(self,uid,mid):
        """
        Predict a user rating on a movie given userID and movieID

        The algorithm is:
                   1. Get index of the provided user id (index_userID)
                   2. Get all the user ratings for the user using index_userID (ratings_index_userID)
                   3. Get index of the provided movie id (index_movieID)
                   4. Get all the similarity scores using index_movieID (movie_sims)
                   5. Take the **averaged** dot product.
        """
        # Predict user rating as follows:
        # 1. Get entry of user id in rating matrix
        # 2. Get entry of movie id in sim matrix
        # 3. Employ 1 and 2 to predict user rating of the movie

        
        # your code here
        uID = self.uid2idx[uid]
        mID = self.mid2idx[mid]
        
        user_ratings = self.Mr[uID]

        simmilarity = self.sim[mID]
        
        return np.dot(simmilarity,user_ratings)/np.dot(simmilarity,user_ratings>0)
    
    def predict(self):
        """
        Predict ratings in the test data. Returns predicted rating in a numpy array of size (# of rows in testdata,)
        """
        # your code here
        yp = [self.predict_from_sim(uId, mId) for uId, mId in zip(self.data.test["uID"],self.data.test["mID"])]
        return np.array(yp)
    
    def rmse(self,yp):
        yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())

    
class ContentBased(RecSys):
    def __init__(self,data):
        super().__init__(data)
        self.data=data
        self.Mm = self.calc_movie_feature_matrix()  
        
    def calc_movie_feature_matrix(self):
        """
        Create movie feature matrix in a numpy array of shape (#allmovies, #genres) 
        """
        # your code here
        movies_feature:pd.DataFrame = self.data.movies.set_index("mID", drop=True).drop(columns=["title","year"])
        
        return  movies_feature.to_numpy()
    
    def calc_item_item_similarity(self):
        """
        Create item-item similarity using Jaccard similarity
        """
        # Update the sim matrix by calculating item-item similarity using Jaccard similarity
        # Jaccard Similarity: J(A, B) = |A∩B| / |A∪B| 
        # your code here
        from tqdm import tqdm

        movies = self.mid2idx.values()
        for i in tqdm(movies):
            for j in movies:
                self.sim[i,j] = 1- jaccard(self.Mm[i],self.Mm[j])
                
class Collaborative(RecSys):    
    def __init__(self,data):
        super().__init__(data)
        
    def calc_item_item_similarity(self, simfunction, *X):  
        """
        Create item-item similarity using similarity function. 
        X is an optional transformed matrix of Mr
        """    
        # General function that calculates item-item similarity based on the sim function and data inputed
        if len(X)==0:
            self.sim = simfunction()            
        else:
            self.sim = simfunction(X[0]) # *X passes in a tuple format of (X,), to X[0] will be the actual transformed matrix
            
    def cossim(self):    
        """
        Calculates item-item similarity for all pairs of items using cosine similarity (values from 0 to 1) on utility matrix
        Returns a cosine similarity matrix of size (#all movies, #all movies)
        """
        # Return a sim matrix by calculating item-item similarity for all pairs of items using Jaccard similarity
        # Cosine Similarity: C(A, B) = (A.B) / (||A||.||B||) 
        # your code here
        
        pass
    
    def jacsim(self,Xr):
        """
        Calculates item-item similarity for all pairs of items using jaccard similarity (values from 0 to 1)
        Xr is the transformed rating matrix.
        """    
        # Return a sim matrix by calculating item-item similarity for all pairs of items using Jaccard similarity
        # Jaccard Similarity: J(A, B) = |A∩B| / |A∪B| 
        # your code here
        
        pass
    
    

# Q1. Baseline models [15 pts]

### 1a. Complete the function `predict_everything_to_3` in the class `RecSys`  [5 pts]

In [134]:
# Creating Sample test data
np.random.seed(42)
sample_train = train[:30000]
sample_test = test[:30000]


sample_MV_users = MV_users[(MV_users.uID.isin(sample_train.uID)) | (MV_users.uID.isin(sample_test.uID))]
sample_MV_movies = MV_movies[(MV_movies.mID.isin(sample_train.mID)) | (MV_movies.mID.isin(sample_test.mID))]


sample_data = Data(sample_MV_users, sample_MV_movies, sample_train, sample_test)

In [135]:
# Sample tests predict_everything_to_3 in class RecSys

sample_rs = RecSys(sample_data)
sample_yp = sample_rs.predict_everything_to_3()
print(sample_rs.rmse(sample_yp))
assert sample_rs.rmse(sample_yp)==approx(1.2642784503423288, abs=1e-3), "Did you predict everything to 3 for the test data?"

1.2642784503423288


### 1b. Complete the function predict_to_user_average in the class RecSys [10 pts]
Hint: Include rated items only when averaging

In [136]:
# Sample tests predict_to_user_average in the class RecSys
sample_yp = sample_rs.predict_to_user_average()
print(sample_rs.rmse(sample_yp))
assert sample_rs.rmse(sample_yp)==approx(1.1429596846619763, abs=1e-3), "Check predict_to_user_average in the RecSys class. Did you predict to average rating for the user?" 

3.2222222222222223
3.0
2.4
2.6
3.5
3.0
3.5
2.3333333333333335
3.6875
3.5652173913043477
3.4
4.333333333333333
4.0
2.935483870967742
4.125
3.4285714285714284
3.6363636363636362
4.0
2.909090909090909
3.5
3.6666666666666665
4.333333333333333
4.888888888888889
3.9
4.0
2.9285714285714284
3.576923076923077
3.4285714285714284
3.8
3.6666666666666665
3.3214285714285716
3.2666666666666666
3.5
3.6666666666666665
4.0
4.0
3.1
4.071428571428571
nan
2.857142857142857
3.5555555555555554
3.4615384615384617
3.6666666666666665
3.6
3.7142857142857144
4.5
3.7058823529411766
3.8666666666666667
4.333333333333333
3.4166666666666665
3.923076923076923
3.2857142857142856
3.925925925925926
3.3333333333333335
nan
4.0
3.857142857142857
nan
3.0
2.962962962962963
4.0
3.1666666666666665
2.6666666666666665
nan
3.5
2.962962962962963
2.0
3.4
2.2
4.5
3.5454545454545454
3.588235294117647
3.642857142857143
3.1333333333333333
3.022222222222222
nan
3.5
3.5
3.7142857142857144
nan
4.0
3.4285714285714284
3.8333333333333335
1.714

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\4259411403.py:44: RuntimeWarning: Mean of empty slice.
  mean_users = { ind: user_ratings[user_ratings>0].mean()  for ind, user_ratings in enumerate(self.Mr)}
c:\Users\OM-FI\Documents\Projects\iml\Unsupervised Algorithms in Machine Learning\Recommender\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Q2. Content-Based model

### 2a. Complete the function calc_movie_feature_matrix in the class ContentBased 

In [137]:
cb = ContentBased(data)
cb.Mm.shape

(3883, 18)

In [138]:
# tests calc_movie_feature_matrix in the class ContentBased 
assert(cb.Mm.shape==(3883, 18))

### 2b. Complete the function calc_item_item_similarity in the class ContentBased [10 pts]
This function updates `self.sim` and does not return a value.    
Some factors to think about:     
1. The movie feature matrix has binary elements. Which similarity metric should be used?
2. What is the computation complexity (time complexity) on similarity calcuation?      
Hint: You may use functions in the `scipy.spatial.distance` module on the dense matrix, but it is quite slow (think about the time complexity). If you want to speed up, you may try using functions in the `scipy.sparse` module. 

In [139]:
# cb.calc_item_item_similarity()

In [140]:
# Sample tests calc_item_item_similarity in ContentBased class 
sample_cb = ContentBased(sample_data)
print(sample_cb.Mm.shape)
sample_cb.calc_item_item_similarity() 

(3152, 18)


100%|██████████| 3152/3152 [01:50<00:00, 28.41it/s]


In [141]:
assert(sample_cb.sim.sum() > 0), "Check calc_item_item_similarity."
assert(np.trace(sample_cb.sim) == 3152), "Check calc_item_item_similarity. What do you think np.trace(cb.sim) should be?"


ans = np.array([[1, 0.25, 0.],[0.25, 1, 0.],[0., 0., 1]])
for pred, true in zip(sample_cb.sim[10:13, 10:13], ans):
    assert approx(pred, 0.01) == true, "Check calc_item_item_similarity. Look at cb.sim"

### 2c. Complete the function predict_from_sim in the class RecSys

In [142]:
# for a, b in zip(sample_MV_users.uID, sample_MV_movies.mID):
#     print(a, b, sample_cb.predict_from_sim(a,b))

# Sample tests for predict_from_sim in RecSys class 
assert(sample_cb.predict_from_sim(245,276)==approx(2.5128205128205128,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."
assert(sample_cb.predict_from_sim(2026,2436)==approx(2.785714285714286,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."

### 2d. Complete the function predict in the class RecSys
After completing the predict method in the RecSys class, run the cell below to calculate rating prediction and RMSE. How much does the performance increase compared to the baseline results from above? 

In [144]:
sample_yp = sample_cb.predict()
sample_rmse = sample_cb.rmse(sample_yp)
print(sample_rmse)

assert(sample_rmse==approx(1.1962537249116723, abs=1e-2)), "Check method predict in the RecSys class."

uID


C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\4259411403.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(simmilarity,user_ratings)/np.dot(simmilarity,user_ratings>0)


1.1962537249116723


Reading Material
1. First, start with a quick revision into terminologies by checking out:  [Quick Revision](https://www.youtube.com/watch?v=4MoSrMkWovM&ab_channel=DataMites)

2. Then, once you are there, let's deep dive into this by following these two videos:

[Deep Dive 1](https://www.youtube.com/watch?v=Blzp9iuhZqo)

[Deep Dive 2](https://www.youtube.com/watch?v=dfMoygb7FZE)

Why we need eigenvectors ? Visualize them [here](https://www.intmath.com/matrices-determinants/eigenvalues-eigenvectors-concept-applet.php).

3. Now, you should be equipped on how to go about Sparse Matrices multiplication and CSE conversions. And, here is everything combined on a Netflix Recommendation System case-study on how to implement a Dot Product for prediction:  [Using Dot Product in a Recommendation Engine](https://youtu.be/ZspR5PZemcs?t=515)

In case you are struggling: Read this excellent [Medium Article](https://medium.com/@chhavi.saluja1401/recommendation-systems-made-simple-b5a79cac8862#:~:text=The%20key%20notion%20here%20is%20to%20determine%20users%2C%20who%20are%20like%20the%20target%20user%20A%2C%20and%20recommend%20ratings%20for%20the%20unobserved%20ratings%20of%20A%20by%20calculating%20weighted%20averages%20of%20the%20ratings%20of%20this%20peer%20group)

